## 全文检索

In [1]:
from pymilvus import MilvusClient, DataType


# Define server address
SERVER_ADDR = "http://localhost:19530"

client = MilvusClient(uri = SERVER_ADDR)

schema = client.create_schema()

schema.add_field(field_name="id", datatype=DataType.INT64, is_primary=True, auto_id=True) 
schema.add_field(field_name="text", datatype=DataType.VARCHAR, max_length=256, enable_analyzer=True)     # VARCHAR 是 Milvus 用于文本存储的字符串数据类型。设置 enable_analyzer=True 以允许 Milvus 对文本进行分词。
schema.add_field(field_name="sparse", datatype=DataType.SPARSE_FLOAT_VECTOR) # 保留用于存储内部生成的稀疏嵌入向量的字段，用于全文搜索操作。数据类型必须为 SPARSE_FLOAT_VECTOR 。

{'auto_id': False, 'description': '', 'fields': [{'name': 'id', 'description': '', 'type': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': True}, {'name': 'text', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 256, 'enable_analyzer': True}}, {'name': 'sparse', 'description': '', 'type': <DataType.SPARSE_FLOAT_VECTOR: 104>}], 'enable_dynamic_field': False}

In [ ]:
# 构建用于全文搜索的 bm25 Function
from langchain_milvus import BM25BuiltInFunction
from pymilvus import FunctionType,Function

bm25_function = Function(
    name = "text_bm25_emb",
    input_field_names=["text"], # # 需要进行文本到稀疏向量转换的 VARCHAR 字段名称。对于 FunctionType.BM25 ，此参数仅接受一个字段名称。
    output_field_names=["sparse"], # # 存储内部生成稀疏向量的字段名称。对于 FunctionType.BM25 ，此参数仅接受一个字段名。
    function_type=FunctionType.BM25 # # 要使用的函数类型。将该值设置为 FunctionType.BM25
)

schema.add_function(bm25_function)

# 配置索引
index_params = client.prepare_index_params()

index_params.add_index(
    field_name="sparse",

    index_type="SPARSE_INVERTED_INDEX",
    metric_type="BM25",
    params={
        "inverted_index_algo": "DAATA_MAXSCORE", # （默认）：采用 MaxScore 算法优化的文档逐次（DAAT）查询处理。
        "bm25_k1": 1.2,       # 数值越高，专业术语词频在文档排名中的重要性越大。取值范围：[1.2, 2.0]。
        "bm25_b": 0.75 
    }
)

# 检查集合是否已存在，如果存在则删除
if client.has_collection("my_collection"):
    client.drop_collection("my_collection")
    print("已删除现有集合 'my_collection'")

# 创建集合collections
client.create_collection(
    collection_name="my_collection",
    schema=schema,
    index_params=index_params,
)

print("成功创建集合 'my_collection'")

ParamError: <ParamError: (code=1, message=The function of schema type must be Function.)>

In [3]:
# 插入数据
client.insert('my_collection', [
    {'text': 'information retrieval is a field of study.'},
    {'text': 'information retrieval focuses on finding relevant information in large datasets.'},
    {'text': 'data mining and information retrieval overlap in research.'},
])

{'insert_count': 3, 'ids': [460357983284649671, 460357983284649672, 460357983284649673], 'cost': 0}

In [6]:
search_params = {
    'params': {
        'drop_ratio_search': 0.2,           # 搜索期间忽略的低重要性术语的比例
    }
}

client.search(
    collection_name="my_collection",
    data=['whats the focus of information retrieval?'], # 自然语言原始查询文本。Milvus 通过 BM25 函数自动将您的文本查询转换为稀疏向量——请勿提供预计算向量。
    search_params=search_params,
    anns_field="sparse", # 包含内部生成稀疏向量的字段名称。 
    output_fields=["id","text"], #'sparse' causes an error
    limit=1,
)

data: [[{'id': 460357983284649671, 'distance': 1.3352930545806885, 'entity': {'text': 'information retrieval is a field of study.', 'id': 460357983284649671}}]]